- which locations need to be visited
- which order should be visited 
- x and y coordintaes
- Parse information, draw routes that are actually created 
- How is the route changing
- Outliers
- City, country side

- Predict how this sequence is changing
    - Which types of routes do we have (clustering)
    - Visualize the routes, do they make sense
    - Developping the model? Making choice of what you gonna do
    - 80 hours (16 hours per person)

In [70]:
# def read_route(request_file_name):
#     return request_file_name.split("-")[0]
# merged['route_id'] = merged['request_file_name'].apply(read_route)

In [71]:
from pyspark.sql import Window, SparkSession
import pyspark.sql.functions as F
import os
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

def read_json(path):
    with open(path) as f:
        d = json.load(f)
        return d

def read_txt(path):
    with open(path) as f:
        return f.read()


def get_files(path):
    return [f for f in listdir(path) if isfile(join(path, f))]


def get_directories(path):
    return [name for name in os.listdir(path) if name != ".DS_Store" and name != "README.txt"]



### Obtain request data frame

In [72]:
requests_folders = "/Users/ehambardzumy/Desktop/big_data/data/requests/"
directories = get_directories(requests_folders)

In [73]:
data_route_day = []
requests_folders = "/Users/ehambardzumy/Desktop/big_data/data/requests/"
directories = get_directories(requests_folders)

for directory in directories:
    route_day_directory = requests_folders+directory
    files_route_day = get_files(route_day_directory)
    for request_json in files_route_day:
        
        request_path = route_day_directory + "/" + request_json
        request_dict = read_json(request_path)
        for indx, route_point in enumerate(request_dict['tasks']):
            timeWindow = route_point['timeWindow']
            address = route_point['address']
            data_point = {**timeWindow, **address, 
                          'id': route_point['id'], 
                          'request_id': request_dict['id'],
                          'configurationName': request_dict['configurationName'],
                          'request_file_name': request_json.split(".")[0]
                         }
            data_route_day.append(data_point)
            
request_full_df = pd.DataFrame(data_route_day)

### Obtain response data frame

In [74]:
data_route_day = []
responses_folders = "/Users/ehambardzumy/Desktop/big_data/data/responses/"
directories = get_directories(responses_folders)

response_dfs = []
for directory in directories:
    route_day_directory = responses_folders+directory
    files_route_day = get_files(route_day_directory)
    for response in files_route_day:
        response_path = route_day_directory + "/" + response
        response_txt = read_txt(response_path)
        
        order_ids = response_txt.split("\n")
        length = len(order_ids)
        response_file_name = [response.split(".")[0]]*length
        response_df = pd.DataFrame({"id":order_ids, 
                                    "request_file_name":response_file_name
                                   })
        
        response_dfs.append(response_df)
response_full_df = pd.concat(response_dfs)

In [75]:
response_full_df.head(5)

,id,request_file_name
0,76805,0521_641-20220616-062619-165-165
1,76809,0521_641-20220616-062619-165-165
2,76850,0521_641-20220616-062619-165-165
3,76755,0521_641-20220616-062619-165-165
4,76759,0521_641-20220616-062619-165-165


### Feature engeneering

In [76]:
request_full_df.head(5)

,from,till,latitude,longitude,id,request_id,configurationName,request_file_name
0,2022-06-16T07:30:00,2022-06-16T23:59:00,0.609201,0.342267,101,fd22a657-774b-4067-b800-94545eb597ad,AddToSequence,0521_641-20220616-191209-168-167
1,2022-06-16T07:30:00,2022-06-16T23:59:00,0.608148,0.339248,100,fd22a657-774b-4067-b800-94545eb597ad,AddToSequence,0521_641-20220616-191209-168-167
2,2022-06-16T07:30:00,2022-06-16T23:59:00,0.607441,0.338553,99,fd22a657-774b-4067-b800-94545eb597ad,AddToSequence,0521_641-20220616-191209-168-167
3,2022-06-16T07:30:00,2022-06-16T23:59:00,0.606546,0.338306,96,fd22a657-774b-4067-b800-94545eb597ad,AddToSequence,0521_641-20220616-191209-168-167
4,2022-06-16T07:30:00,2022-06-16T23:59:00,0.606498,0.338540,97,fd22a657-774b-4067-b800-94545eb597ad,AddToSequence,0521_641-20220616-191209-168-167


In [77]:
response_full_df.shape[0], request_full_df.shape[0]

(2572035, 2576770)

### Merge then so that the ids are in the same order as response

In [78]:
merged = pd.merge(response_full_df, 
                  request_full_df, 
                  on = ["request_file_name", "id"])

#### !!! Loss of couple of data points

In [79]:
merged.shape[0]

2572033

In [80]:
merged.head(5)

,id,request_file_name,from,till,latitude,longitude,request_id,configurationName
0,76805,0521_641-20220616-062619-165-165,2022-06-16T07:30:00,2022-06-16T23:59:00,0.535067,0.371578,c10b2cc7-b21e-4f38-b4d0-5fcbf52e9f86,EstimateTime
1,76809,0521_641-20220616-062619-165-165,2022-06-16T07:30:00,2022-06-16T23:59:00,0.535132,0.372101,c10b2cc7-b21e-4f38-b4d0-5fcbf52e9f86,EstimateTime
2,76850,0521_641-20220616-062619-165-165,2022-06-16T07:30:00,2022-06-16T23:59:00,0.535745,0.374195,c10b2cc7-b21e-4f38-b4d0-5fcbf52e9f86,EstimateTime
3,76755,0521_641-20220616-062619-165-165,2022-06-16T07:30:00,2022-06-16T23:59:00,0.536241,0.375161,c10b2cc7-b21e-4f38-b4d0-5fcbf52e9f86,EstimateTime
4,76759,0521_641-20220616-062619-165-165,2022-06-16T07:30:00,2022-06-16T23:59:00,0.536879,0.375186,c10b2cc7-b21e-4f38-b4d0-5fcbf52e9f86,EstimateTime


In [81]:
list(set(merged.configurationName))

['AddToSequence', 'CreateSequence', 'EstimateTime']

1. Group by request_file_name
2. Compute route length (number of points) 
3. Average euclidean distance between consecutive points
3. Maximum euclidean distance 
4. Total distance

5. the removed point has a reasoning behing to be discovered
   - Average distance of the point relative to other points as
   - The point must be removed if exceeding certain distance treshold 
   - How many estimate times were needed to remove a point

## Route level metrics
- Route length - number of points
- Average euclidean distance between consecutive points
- Maximum euclidean distance
- Total distance
- How many estiamte times were performed
- Should we drop duplicate estimate times? 
    - (in the sense that the number of tasks has not changed)
- 

In [83]:
merged['latitude_next'] = (merged.groupby(['request_file_name'])['latitude'].shift(-1))
merged['longitude_next'] = (merged.groupby(['request_file_name'])['longitude'].shift(-1))

merged.to_csv("./merged.csv", index=False)